In [1]:
import os

cwd = os.getcwd()
os.chdir('/home/a-m/marri2/visual_servoing/attempt3')

In [2]:
#imports
import glob
import os
import numpy as np
import random
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import math

from PIL import Image
import torch
torch.cuda.empty_cache()
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
from torchvision import models
from torchvision.transforms import ToTensor, Normalize
import torch.optim as optim
import logging
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, utils, transforms
import re
from torch.nn import ReplicationPad3d

In [3]:
# global variable
device = torch.device("cuda:0")

In [4]:
class ImagePoseDataset(Dataset):
    
    #reading csv file with labels
    def __init__(self, image_dir, size = 600, transform=None):
        annotation_files = glob.glob(image_dir + '/annotated_*.csv')
        
        assert len(annotation_files) == 1, "None or more than one annotation file found at " + image_dir
        self.annotations = pd.read_csv(annotation_files[0])
        
        self.annotations.astype({'B': 'float32','R': 'float32'}).dtypes  #change this
        
        self.data = glob.glob(image_dir + '/**/*.jpg', recursive=True)
        self.data.sort(key=lambda f: int(re.sub('\D', '', f)))
    
        
        unequal_error = "Number of images not equal to number of annotations: {} != {}".format(len(self.data), len(self.annotations["image"].values))
        assert len(self.data) == len(self.annotations["image"].values), unequal_error
        

        # target_scaler = MinMaxScaler()
        # target_scaler.fit(y_train)
        # y_train = target_scaler.transform(y_train)
        # y_test = target_scaler.transform(y_test)

        self.image_dir = image_dir
        self.image_size = [224, 244] #change this
        self.transform = transform
        self.size = size
        self.minb = self.annotations["B"].values.min()
        self.minr = self.annotations["R"].values.min()
        self.maxb = self.annotations["B"].values.max()
        self.maxr = self.annotations["R"].values.max()
        self.pairs = []
        
        for i in range(len(self.data)):
            for j in np.random.choice(range(len(self.data)), size = 1, replace=False):
                self.pairs.append((i,j))

        
    def __len__(self):        
        return len(self.pairs)

    def __getitem__(self, idx):
        np.random.seed(230)
        
        first_img_path = self.data[self.pairs[idx][0]]
        second_img_path = self.data[self.pairs[idx][1]]
        
        img_paths = [first_img_path, second_img_path]
        img_name = [os.path.split(img_paths[0])[-1][:-4], os.path.split(img_paths[1])[-1][:-4]]  #might have to change this
        images = [Image.open(first_img_path), Image.open(second_img_path)]
        
        
        pose = np.zeros((1, 2), dtype=np.float32)  #change this
        annotation1 = self.annotations.loc[self.annotations["image"] == img_name[0]].iloc[0]
        annotation2 = self.annotations.loc[self.annotations["image"] == img_name[1]].iloc[0]
    
        ann1 = annotation1[2:].to_numpy()
        ann2 = annotation2[2:].to_numpy()
    
        pose[0] = ann1 - ann2
      #  pose[1] = ann2 - ann1
        pose[0][0] = (pose[0][0] - self.minb)/(self.maxb - self.minb)
        pose[0][1] = (pose[0][1] - self.minr)/(self.maxr - self.minr)
        # print(pose)
                
        for i in range(len(images)):            
            if i == 0:
                j = torch.unsqueeze(self.transform(images[i]), dim=0)
            else:
                k = torch.unsqueeze(self.transform(images[i]), dim=0)
                j = torch.cat((j,k), dim=0)
        data = j
        
 
        sample = {"images": data , "poses": pose, "filename":img_name}        
        
        return sample


def fetch_dataloader(types, data_dir, batch_size, num_workers):
    """
    Fetches the DataLoader object for each type in types from data_dir.

    Args:
        types: (list) has one or more of 'train', 'val', 'test' depending on which data is required
        data_dir: (string) directory containing the dataset
        params: (Params) hyperparameters

    Returns:
        data: (dict) contains the DataLoader object for each type in types
    """
    dataloaders = {}
    
    img_dim = 224

    train_transforms = transforms.Compose([transforms.Resize((img_dim, img_dim), Image.BICUBIC),
                                           transforms.ToTensor(),
                                           transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]) 
    
    test_transform = transforms.Compose([transforms.Resize((img_dim, img_dim), Image.BICUBIC),
                                         transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]) 
    
    for split in ['train', 'val', 'test']:
        if split in types:
            path = os.path.join(data_dir, split)

            if split == 'train':
                dl = DataLoader(ImagePoseDataset(path, size = 100, transform = train_transforms), 
                                        batch_size=batch_size, shuffle=False,
                                        num_workers=num_workers)

            else:
                dl = DataLoader(ImagePoseDataset(path, size = 10, transform = test_transform), 
                                batch_size=batch_size, shuffle=False,
                                num_workers=num_workers)

            dataloaders[split] = dl

    return dataloaders


In [5]:
# data_dir = 'theta-biocluster'
# train_dataloader = fetch_dataloader(['train'], data_dir, batch_size=4,num_workers=0)['train']

In [6]:

class PoseModel(nn.Module):

    def __init__(self):
        super(PoseModel, self).__init__()
        model1 = models.resnet50(pretrained=True)
        model2 = models.resnet50(pretrained=True)
        
        self.enc1 = nn.Sequential(*list(model1.children())[:-2])
        self.enc2 = nn.Sequential(*list(model2.children())[:-2])       
        
        self.conv1 = nn.Conv2d(4096, 2048, kernel_size=(3,3), stride=(2,2), padding=(1,1))
        self.bn1 = nn.BatchNorm2d(2048)
        
        self.conv2 = nn.Conv2d(2048, 1024, kernel_size=(3,3), stride=(2,2), padding=(1,1))  
        self.bn2 = nn.BatchNorm2d(1024)

        self.conv3 = nn.Conv2d(1024, 512, kernel_size=(3,3), stride=(2,2), padding=(1,1))  
        self.bn3 = nn.BatchNorm2d(512)

        self.conv4 = nn.Conv2d(512, 256, kernel_size=(3,3), stride=(2,2), padding=(1,1))  
        self.bn4 = nn.BatchNorm2d(256)

        self.fc1 = nn.Linear(512, 128)
        self.out= nn.Linear(128, 2)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, s): 
        
        feature1 = self.enc1(s[:,0])
        feature2 = self.enc2(s[:,1])
        feature = torch.cat([feature1,feature2],1)
        print(feature.shape)
        x = F.relu(self.bn1(self.conv1(feature)))
        print(x.shape)
        x = F.relu(self.bn2(self.conv2(x)))
        print(x.shape)
        x = F.relu(self.bn3(self.conv3(x)))
        print(x.shape)
        # x = F.relu(self.bn4(self.conv4(x)))
        print(x.shape)
        x = torch.reshape(x, (x.size(0),-1))
        print(x.shape)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.out(x)
        
        x = x.view(s.size(0), 1, 2) 

        return x
        

net = PoseModel()


dummy_input = torch.rand((2, 2, 3, 224, 244))
output = net(dummy_input)

# print(net)
print(output.shape)

torch.Size([2, 4096, 7, 8])
torch.Size([2, 2048, 4, 4])
torch.Size([2, 1024, 2, 2])
torch.Size([2, 512, 1, 1])
torch.Size([2, 512, 1, 1])
torch.Size([2, 512])
torch.Size([2, 1, 2])


In [7]:
class MyCriterion(nn.Module):
    def __init__(self):
        super(MyCriterion, self).__init__()        

    def forward(self, prediction, target): 
            
        return F.mse_loss(prediction, target, reduction="mean")
    
def show_memory():
    t = torch.cuda.get_device_properties(0).total_memory
    r = torch.cuda.memory_reserved(0) 
    a = torch.cuda.memory_allocated(0)
    f = (r-a)  # free inside reserved
    print(np.array([t, r, a, f])/(1024*1024))

torch.cuda.empty_cache()        
show_memory()

[11178.5     0.      0.      0. ]


In [8]:
def simple_train(model, criterion, optimizer, train_dataloader, **kwargs):
    model.train()
    losses = []
    
    torch.cuda.empty_cache()        
    show_memory()
    
    for i, batch in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        img, gt = batch['images'], batch['poses']     
        img = img.to(device)
        
     #   print(img.shape)
        gt = gt.to(device)
        
        img = img.float()
        pred = model(img)
        print(pred.shape, gt.shape)
        
        loss = criterion.forward(pred,gt)
      
        losses.append(loss.item())
        loss.backward()
        optimizer.step()

        show_memory()
        del img
        del gt
        torch.cuda.empty_cache()        
        show_memory()

    return sum(losses)/len(losses)
  



def simple_predict(dataloader, model):
    model.eval()    
   
    with torch.no_grad():
        for i, batch in enumerate(tqdm(dataloader)):
            
            img, gt = batch['images'], batch['poses']  
            img = img.to(device)
            gt = gt.to(device)
            img = img.float()
            
            pred = model(img)
            # print(pred)

            pred = pred.view(-1,3)
            gt =  gt.view(-1,3)
            
            loss = torch.mean(F.mse_loss(pred, gt, reduction="none"),0)
            
            if i==0:
                losses = torch.clone(torch.unsqueeze(loss,0))
                
            losses = torch.cat([losses,torch.unsqueeze(loss,0)],0)
    
    losses = torch.mean(losses, 0).cpu().numpy()
    b, r, theta = losses[0],losses[1], losses[2]
    
    

    return np.mean(losses),losses[0],losses[1], losses[2], pred

In [9]:

num_epochs = 50
batch_size = 4


model = PoseModel().to(device)


criterion = MyCriterion().to(device)
parameter = model.parameters()
optimizer = optim.Adam(parameter, lr=1e-4, weight_decay=1e-2)

# #scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[4], gamma=0.1)
data_dir = 'data'
train_dataloader = fetch_dataloader(['train'], data_dir, batch_size=batch_size,num_workers=0)['train']
val_dataloader = fetch_dataloader(['val'], data_dir, batch_size=1,num_workers=0)['val']
test_dataloader = fetch_dataloader(['test'], data_dir, batch_size=1,num_workers=0)['test']
print(len(train_dataloader))


train_losses = []
val_losses = []
b1 = []
r1 = []
theta1 = []
x1 = []
y1 = []

predicts = []

best_loss = 5000
for epoch in range(num_epochs):
  
    
    print("Epoch {}/{}".format(epoch+1, num_epochs))

    epoch_loss = simple_train(model, criterion, optimizer, train_dataloader)
    train_losses.append(epoch_loss)
    
    print("Training loss {}".format(epoch_loss))
    # print("Training metrics: b error %.4f, r error %.4f, theta error %.4f, x error %.4f, y error %.4f" % (train_losses[0],train_losses[1],train_losses[2], train_losses[3], train_losses[4]))

    val_loss, b, r, theta, pred = simple_predict(val_dataloader, model)
    b1.append(b)
    r1.append(r)
    theta1.append(theta)
    
    predicts.append(pred)
    val_losses.append(val_loss)

    print("Validation loss :", val_loss)
    print("Validation metrics: b error %.4f, r error %.4f , theta error %.4f" % (b, r, theta))

    print("predictions:", pred)

   
    if val_loss<best_loss:          
        torch.save(model.state_dict(), 'best_model_theta_outputs.torch')         
        best_loss = val_loss
        
    #scheduler.step()




  0%|          | 0/70 [00:00<?, ?it/s]

70
Epoch 1/50
[11178.5          594.           565.40039062    28.59960938]


  1%|▏         | 1/70 [00:00<00:46,  1.48it/s]

torch.Size([4, 4096, 7, 7])
torch.Size([4, 2048, 4, 4])
torch.Size([4, 1024, 2, 2])
torch.Size([4, 512, 1, 1])
torch.Size([4, 512, 1, 1])
torch.Size([4, 512])
torch.Size([4, 1, 2]) torch.Size([4, 1, 2])
[11178.5         3394.          2260.14013672  1133.85986328]
[11178.5         2314.          2255.54589844    58.45410156]


  3%|▎         | 2/70 [00:01<00:42,  1.61it/s]

...

  0%|          | 0/80 [00:00<?, ?it/s]

torch.Size([4, 4096, 7, 7])
torch.Size([4, 2048, 4, 4])
torch.Size([4, 1024, 2, 2])
torch.Size([4, 512, 1, 1])
torch.Size([4, 512, 1, 1])
torch.Size([4, 512])
torch.Size([4, 1, 2]) torch.Size([4, 1, 2])
[11178.5         3912.          2260.14013672  1651.85986328]
[11178.5         2322.          2255.54589844    66.45410156]
Training loss 0.22635758950241974


  0%|          | 0/80 [00:00<?, ?it/s]


torch.Size([1, 4096, 7, 7])
torch.Size([1, 2048, 4, 4])
torch.Size([1, 1024, 2, 2])
torch.Size([1, 512, 1, 1])
torch.Size([1, 512, 1, 1])
torch.Size([1, 512])


RuntimeError: shape '[-1, 3]' is invalid for input of size 2

In [ ]:
print(train_losses)

In [ ]:
print(val_losses)

In [ ]:
print(predicts)

In [ ]:
# metrics = predicts

In [ ]:
print([[b1[i],r1[i],theta1[i],x1[i],y1[i]] for i in range(len(b1))])

In [ ]:
# metrics = np.asarray(metrics)

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# fig, ax1 = plt.subplots()
# color = 'tab:red'
# ax1.set_xlabel('Epochs')
# ax1.set_ylabel('Error (cm)', color=color)
# ax1.plot(epochs, metrics[:,0], 'r')
# ax1.plot(epochs, metrics[:,1], 'b')
# ax1.plot(epochs, metrics[:,2], 'g')
# ax1.tick_params(axis='y', labelcolor=color)
# ax1.set_ylim(0, 0.1)
# ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
# color = 'tab:blue'
# ax2.set_ylabel('Error (radians)', color=color)  # we already handled the x-label with ax1
# ax2.plot(epochs, metrics[:,3], 'y')
# ax2.plot(epochs, metrics[:,4], 'c')
# ax2.tick_params(axis='y', labelcolor=color)
# fig.tight_layout()  # otherwise the right y-label is slightly clipped
# plt.show()

In [ ]:
# predicts1 = [[71.3455, 299.98846, 612.14197, 17.913647, 67.760956], [70.61943, 308.3565, 614.71674, 18.341103, 66.79955], [72.254776, 308.16354, 624.0446, 17.517784, 68.64179], [79.20256, 313.67682, 756.6699, 21.187294, 76.29698], [68.64581, 445.8373, 1308.4067, 45.08603, 147.08012], [67.010864, 394.3578, 1163.5549, 37.64315, 111.245186], [72.79684, 407.45255, 1438.2988, 37.703526, 122.95048], [66.26516, 509.4921, 2232.0396, 53.581795, 160.51176], [71.617165, 553.7531, 2251.1345, 50.06454, 163.87398], [67.6314, 645.1201, 3004.0298, 64.6395, 201.75027], [68.610435, 670.9459, 3495.046, 70.110756, 229.66035], [78.080925, 838.47186, 3762.1555, 78.345604, 224.2315], [82.31957, 791.64526, 3300.3245, 68.491196, 200.6706], [85.77867, 724.0587, 2986.078, 61.731506, 186.24583], [88.41423, 767.7903, 2660.4978, 58.266937, 180.84425], [92.60425, 729.06586, 2681.3442, 55.219467, 174.54987], [88.064064, 702.8218, 2273.686, 47.222137, 142.69472], [92.22376, 915.272, 3502.126, 67.31765, 197.83511], [95.04986, 732.945, 2337.5552, 47.52399, 143.4624], [103.41726, 750.2705, 1805.5945, 39.52808, 141.29509], [115.16615, 639.1609, 1685.0542, 31.669968, 111.18192], [104.19611, 698.6576, 1724.9683, 36.858562, 113.58083], [112.40713, 964.80774, 2499.877, 51.554733, 155.13557], [95.69072, 626.71136, 1799.6161, 37.875294, 117.63456], [128.13573, 694.4404, 2200.0706, 35.31282, 113.16661], [115.73999, 649.15173, 1381.557, 29.185846, 106.21257], [132.67511, 638.09875, 1506.5328, 31.595715, 90.02608], [140.15654, 648.0216, 1620.7126, 32.83686, 100.44681], [133.61388, 673.5364, 1540.6757, 31.223743, 103.85487], [120.989136, 620.9135, 2077.5657, 36.32834, 108.56132], [132.84982, 589.97534, 1586.4774, 31.332727, 85.892555], [134.00136, 630.5227, 2079.9744, 37.17443, 105.96474], [142.30118, 713.86334, 1812.0747, 34.262226, 93.07108], [131.9562, 618.09344, 1308.4608, 30.623444, 92.09023], [150.78465, 771.7664, 1536.8103, 38.427544, 88.707756], [124.79615, 650.9295, 1969.3282, 37.62735, 109.81103], [117.48657, 528.9835, 1442.0688, 30.423353, 84.737495], [166.30316, 554.52606, 1387.5977, 27.794542, 72.16256], [132.65923, 610.9276, 1272.4724, 30.719393, 83.64812], [170.80846, 651.6129, 1587.1293, 31.822659, 86.05595], [148.26839, 546.7099, 1423.1058, 23.101112, 69.18983], [119.36049, 562.3412, 1525.4735, 33.434387, 90.121826], [126.641014, 579.34705, 1643.3766, 42.385773, 106.71667], [153.49957, 608.18365, 1292.7495, 30.102694, 72.05002], [140.59573, 538.986, 1275.6699, 26.956669, 72.718094], [128.63644, 565.2077, 1283.3916, 31.26406, 80.66515], [142.91138, 589.8891, 1425.4674, 36.772514, 108.61162], [174.99702, 604.99945, 1231.4559, 25.457443, 73.68763], [124.46342, 532.1797, 1284.4299, 37.867233, 90.98839], [156.39485, 572.2624, 1304.4873, 29.487886, 64.113655], [141.00058, 599.98944, 1427.2222, 32.966682, 78.49583], [140.62863, 510.04877, 1460.5568, 28.990866, 70.09744], [122.61561, 541.0749, 1227.1143, 30.144491, 77.38699], [118.877815, 520.6939, 1433.2882, 35.451847, 67.93618], [149.10368, 488.75287, 1207.9657, 27.233482, 74.68739], [114.91516, 574.0989, 1348.9976, 40.59728, 87.81351], [123.55287, 516.64185, 1131.8646, 32.938347, 82.748245], [153.83464, 508.87488, 1146.1227, 26.891317, 54.73151], [140.56522, 608.0974, 1416.4187, 40.215973, 79.94072], [152.53659, 592.6191, 1324.523, 39.891792, 78.46605]]


In [ ]:
# predicts1 = np.asarray(predicts1)

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt

# epochs = range(1,61)
# fig, ax1 = plt.subplots()

# color = 'tab:red'
# ax1.set_xlabel('Epochs')
# ax1.set_ylabel('Error (cm)', color=color)
# ax1.plot(epochs, predicts1[:,0], 'r',)
# ax1.plot(epochs, predicts1[:,1], 'b')
# ax1.plot(epochs, predicts1[:,2], 'g')
# ax1.tick_params(axis='y', labelcolor=color)
# # ax1.set_ylim(0, 0.1)

# ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

# color = 'tab:blue'
# ax2.set_ylabel('Error (radians)', color=color)  # we already handled the x-label with ax1
# ax2.plot(epochs, predicts1[:,3], 'm')
# ax2.plot(epochs, predicts1[:,4], 'c')
# ax2.tick_params(axis='y', labelcolor=color)
# plt.title('x,y,z,phi,theta,psi error')
# fig.tight_layout()  # otherwise the right y-label is slightly clipped
# plt.show()

In [ ]:
# epochs = range(1,41)
# plt.plot(epochs, train_losses, 'g', label='Training loss')
# plt.plot(epochs, val_losses, 'b', label='validation loss')
# plt.title('Training and Validation loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

In [ ]:
# metrics = [[27.24967, 313.78604, 103.22942, 0.12602033, 2.4108167], [27.305887, 312.19617, 103.47133, 0.14717528, 2.3978324], [27.524355, 311.05536, 104.642555, 0.15902913, 2.4331577], [27.548677, 325.57285, 106.00729, 0.13429697, 2.4158552], [27.098442, 328.60037, 104.91552, 0.12587117, 2.472294], [27.359726, 334.21838, 104.05903, 0.12709449, 2.4215572], [27.339266, 358.40887, 106.68851, 0.11997138, 2.393511], [27.588818, 383.73975, 107.74374, 0.11348984, 2.3892288], [27.389458, 371.63693, 105.884705, 0.12097337, 2.403273], [27.360971, 342.81848, 106.109245, 0.11562351, 2.3738363], [27.719664, 393.87692, 110.986465, 0.119660616, 2.3882978], [27.633759, 339.6035, 105.70057, 0.113286234, 2.3775778], [28.07819, 360.8138, 112.83532, 0.11849985, 2.3681376], [28.364185, 360.29456, 117.695984, 0.113442786, 2.388075], [29.205482, 357.27008, 125.467896, 0.11830383, 2.386264], [28.379833, 344.7701, 112.977585, 0.10953671, 2.3661098], [28.847458, 355.65115, 118.61085, 0.111811385, 2.4001744], [27.850155, 341.3711, 114.77934, 0.10915208, 2.3725796], [27.88572, 346.0621, 118.30813, 0.110895686, 2.3910563], [27.872988, 349.0645, 115.55186, 0.10912815, 2.3672535], [28.218578, 353.3147, 131.556, 0.11358704, 2.3951552], [28.392235, 339.58942, 121.12667, 0.111038156, 2.3803837], [28.25236, 329.97418, 115.96446, 0.10942464, 2.3864708], [29.406773, 366.15808, 123.42579, 0.11040557, 2.374963], [29.454271, 344.52792, 131.52682, 0.11232563, 2.3724134], [30.187252, 347.98532, 116.14241, 0.10585186, 2.3729453], [29.41112, 349.68896, 114.18263, 0.10918645, 2.3731532], [28.895391, 348.74268, 113.15022, 0.10991674, 2.3822908], [29.371452, 335.8846, 115.51087, 0.11001146, 2.3862026], [31.134686, 344.45355, 119.50881, 0.108406834, 2.375381], [30.70894, 342.97552, 120.56805, 0.11375582, 2.3945308], [29.832468, 343.45724, 109.87155, 0.10899758, 2.3766632], [29.862116, 343.1723, 113.25473, 0.10866166, 2.3900619], [30.768106, 342.69632, 112.46852, 0.10878944, 2.376799], [31.60865, 343.18985, 121.2012, 0.11096387, 2.392029], [31.704603, 339.0673, 129.9622, 0.109866515, 2.3703597], [29.185385, 334.6232, 109.926186, 0.10787476, 2.377635], [30.564453, 349.78403, 112.91635, 0.10928899, 2.3760023], [30.50954, 335.69626, 113.32439, 0.10892574, 2.3779461], [30.807068, 329.50632, 111.41365, 0.108789176, 2.380784]]

In [ ]:
# metrics = np.asarray(metrics)

In [ ]:
# import numpy as np
# # import matplotlib.pyplot as plt

# epochs= range(1, 41)
# fig, ax1 = plt.subplots()

# color = 'tab:red'
# ax1.set_xlabel('Epochs')
# ax1.set_ylabel('Error (cm)', color=color)
# ax1.plot(epochs, metrics[:,0], 'r',)
# ax1.plot(epochs, metrics[:,1], 'b')
# ax1.plot(epochs, metrics[:,2], 'g')
# ax1.tick_params(axis='y', labelcolor=color)
# # ax1.set_ylim(0, 0.1)

# ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

# color = 'tab:blue'
# ax2.set_ylabel('Error (radians)', color=color)  # we already handled the x-label with ax1
# ax2.plot(epochs, metrics[:,3], 'm')
# ax2.plot(epochs, metrics[:,4], 'c')
# # ax2.plot(epochs, metrics[:,5], 'y')
# ax2.tick_params(axis='y', labelcolor=color)
# plt.title('x,y,z,phi,theta,psi error')
# fig.tight_layout()  # otherwise the right y-label is slightly clipped
# plt.show()